In [8]:
import psycopg2

In [9]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=alpha")
except psycopg2.Error as e:
    print('Could not make connection to the Postgre database')
    print(e)

In [10]:
try:
    cursor = conn.cursor()
except psycopg2.Error as e:
    print('Could not create a cursor to the database')
    print(e)

In [11]:
conn.set_session(autocommit=True)

In [12]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MUSIC_LIBRARY (
            ALBUM_ID INT,
            ALBUM_NAME VARCHAR,
            ARTIST_NAME VARCHAR,
            YEAR INT,
            SONGS TEXT[]
        );
    ''')
except psycopg2.Error as e:
    print('Could not create table')
    print(e)

In [13]:
query = '''
    INSERT INTO MUSIC_LIBRARY (
        ALBUM_ID,
        ALBUM_NAME,
        ARTIST_NAME,
        YEAR,
        SONGS
    ) 
    VALUES (
        %s, %s, %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Rubber Soul', 'The Beatles', 1965, ['Michele', 'Think For Yourself', 'In My Life']))
    cursor.execute(query, (2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across The Universe']))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [17]:
try:
    cursor.execute('SELECT * FROM MUSIC_LIBRARY')
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [15]:
# Method 1 for fetching
for row in cursor.fetchall():
    print(row)

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michele', 'Think For Yourself', 'In My Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across The Universe'])


In [18]:
# Method 2 for fetching
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michele', 'Think For Yourself', 'In My Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across The Universe'])


<h3>Starting to convert the database into the 1NF</h3>
<h4>No sets or lists should be present in the database</h4>


In [19]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS MUSIC_LIBRARY_NUCLEAR (
            ALBUM_ID INT,
            ALBUM_NAME VARCHAR,
            ARTIST_NAME VARCHAR,
            YEAR INT,
            SONGS VARCHAR
        );
    ''')
except psycopg2.Error as e:
    print('Could not create table')
    print(e)

In [21]:
query = '''
    INSERT INTO MUSIC_LIBRARY_NUCLEAR (
        ALBUM_ID,
        ALBUM_NAME,
        ARTIST_NAME,
        YEAR,
        SONGS
    ) 
    VALUES (
        %s, %s, %s, %s, %s
    ) 
'''
try:
    for song in ['Michele', 'Think For Yourself', 'In My Life']:
        cursor.execute(query, (1, 'Rubber Soul', 'The Beatles', 1965, song))
    for song in ['Let It Be', 'Across The Universe']:
        cursor.execute(query, (2, 'Let It Be', 'The Beatles', 1970, song))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [22]:
try:
    cursor.execute('SELECT * FROM MUSIC_LIBRARY_NUCLEAR')
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)
    
# Method 2 for fetching
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(1, 'Rubber Soul', 'The Beatles', 1965, 'Michele')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let It Be', 'The Beatles', 1970, 'Let It Be')
(2, 'Let It Be', 'The Beatles', 1970, 'Across The Universe')


<h3>Starting to convert the database into the 2NF</h3>
<h4>Every table must have a primary key</h4>

In [40]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ALBUM_LIBRARY (
            ALBUM_ID INT,
            ALBUM_NAME VARCHAR,
            ARTIST_NAME VARCHAR,
            YEAR INT,
            PRIMARY KEY (ALBUM_ID)
        );
    ''')
except psycopg2.Error as e:
    print('Could not create album table')
    print(e)

In [41]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SONG_LIBRARY (
            SONG_ID INT,
            SONG_NAME VARCHAR,
            ALBUM_ID INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song table')
    print(e)

In [ ]:
query = '''
    INSERT INTO ALBUM_LIBRARY (
        ALBUM_ID,
        ALBUM_NAME,
        ARTIST_NAME,
        YEAR
    ) 
    VALUES (
        %s, %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Rubber Soul', 'The Beatles', 1965))
    cursor.execute(query, (2, 'Let It Be', 'The Beatles', 1970))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [43]:
query = '''
    INSERT INTO SONG_LIBRARY (
        SONG_ID,
        SONG_NAME,
        ALBUM_ID
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Michele', 1))
    cursor.execute(query, (2, 'Think For Yourself', 1))
    cursor.execute(query, (3, 'In My Life', 1))
    cursor.execute(query, (4, 'Let It Be', 2))
    cursor.execute(query, (5, 'Across The Universe', 2))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [50]:
query = '''
    SELECT 
          A.SONG_NAME
        , B.*
    
    FROM 
        SONG_LIBRARY AS A 
            LEFT JOIN 
                ALBUM_LIBRARY AS B ON A.ALBUM_ID = B.ALBUM_ID
'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('Michele', 1, 'Rubber Soul', 'The Beatles', 1965)
('Think For Yourself', 1, 'Rubber Soul', 'The Beatles', 1965)
('In My Life', 1, 'Rubber Soul', 'The Beatles', 1965)
('Let It Be', 2, 'Let It Be', 'The Beatles', 1970)
('Across The Universe', 2, 'Let It Be', 'The Beatles', 1970)


<h3>Starting to convert the database into the 3NF</h3>
<h4>Separate out transitive properties</h4>

In [51]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ALBUM_LIBRARY_3NF (
            ALBUM_ID INT,
            ALBUM_NAME VARCHAR,
            ARTIST_ID INT,
            YEAR INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create album table')
    print(e)

In [52]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SONG_LIBRARY_3NF (
            SONG_ID INT,
            SONG_NAME VARCHAR,
            ALBUM_ID INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song table')
    print(e)

In [53]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ARTIST_LIBRARY_3NF (
            ARTIST_ID INT,
            ARTIST_NAME VARCHAR
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song table')
    print(e)

In [54]:
query = '''
    INSERT INTO ALBUM_LIBRARY_3NF (
        ALBUM_ID,
        ALBUM_NAME,
        ARTIST_ID,
        YEAR
    ) 
    VALUES (
        %s, %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Rubber Soul', 1, 1965))
    cursor.execute(query, (2, 'Let It Be', 1, 1970))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [55]:
query = '''
    INSERT INTO SONG_LIBRARY_3NF (
        SONG_ID,
        SONG_NAME,
        ALBUM_ID
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Michele', 1))
    cursor.execute(query, (2, 'Think For Yourself', 1))
    cursor.execute(query, (3, 'In My Life', 1))
    cursor.execute(query, (4, 'Let It Be', 2))
    cursor.execute(query, (5, 'Across The Universe', 2))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [56]:
query = '''
    INSERT INTO ARTIST_LIBRARY_3NF (
        ARTIST_ID,
        ARTIST_NAME
    ) 
    VALUES (
        %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'The Beatles'))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [57]:
query = '''
    SELECT 
          A.SONG_NAME
        , B.*
        , C.ARTIST_NAME
    FROM 
        SONG_LIBRARY_3NF AS A 
            LEFT JOIN 
                ALBUM_LIBRARY_3NF AS B ON A.ALBUM_ID = B.ALBUM_ID
            LEFT JOIN 
                ARTIST_LIBRARY_3NF AS C ON B.ARTIST_ID = C.ARTIST_ID
'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('Michele', 1, 'Rubber Soul', 1, 1965, 'The Beatles')
('Think For Yourself', 1, 'Rubber Soul', 1, 1965, 'The Beatles')
('In My Life', 1, 'Rubber Soul', 1, 1965, 'The Beatles')
('Let It Be', 2, 'Let It Be', 1, 1970, 'The Beatles')
('Across The Universe', 2, 'Let It Be', 1, 1970, 'The Beatles')
